In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import dstack

# Data transformation
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

# Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

#ClassifiersP
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers import KerasClassifier

# Warnings deactivate
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (8, 6)

pd.set_option("precision", 2)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df = pd.read_csv('Bank-Data/bank-additional-full.csv')

In [ ]:
# Transform to categorical data
col_cat = list(df.select_dtypes(include=['object']).columns)
df.loc[:, col_cat] = df[col_cat].astype('category')

In [ ]:
# Create DataSets
X = df.iloc[:,:-1]  #input columns
y = df.iloc[:,-1]   #target column 

In [ ]:
# Encoding
col_cat.pop()
oe = OrdinalEncoder()
oe.fit(X[col_cat])

X_cat_enc = oe.transform(X[col_cat])
X_cat_enc = pd.DataFrame(X_cat_enc)
X_cat_enc.columns = col_cat

In [ ]:
# Normalization
col_num = list(df.select_dtypes(include =['int64', 'float64']).columns)
scaler = MinMaxScaler(feature_range=(0, 1))
X_num_enc = scaler.fit_transform(X[col_num])
X_num_enc = pd.DataFrame(X_num_enc)
X_num_enc.columns = col_num
x_enc = pd.concat([X_cat_enc, X_num_enc], axis=1)

In [ ]:
# Encoding target
le = LabelEncoder()
le.fit(y)
y_enc = le.transform(y)
y_enc = pd.Series(y_enc)
y_enc.columns = y.name

In [ ]:
# Remove correlated fields
col = list(x_enc.columns)
col.remove('emp.var.rate')
col.remove('nr.employed')
x_enc = x_enc[col]

In [ ]:
# Feature selection
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(x_enc,y_enc)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x_enc.columns)
featureScores = pd.concat([dfcolumns, dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
x_enc = x_enc[list(featureScores.nlargest(10,'Score')['Specs'])]

print(x_enc)
print(y_enc)